In [2]:
import pandas as pd

In [9]:
from surprise import accuracy,SVD,Dataset,Reader
from surprise.model_selection import train_test_split,cross_validate

# 데이터 불러오기

In [3]:
rat=pd.read_csv('F:/PythonStudy/캐글 활용/data/ml-latest-small/ratings.csv')

In [4]:
rat

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


## reader 파라미터 설정

In [10]:
reader=Reader(rating_scale=(0.5,5))
data=Dataset.load_from_df(rat[['userId','movieId','rating']],reader)

## 교차검증 메서드 사용 모델 성능 확인

In [12]:
svd=SVD(random_state=0)
cross_validate(svd,data,measures=('RMSE','MSE'),cv=5,verbose=True)

Evaluating RMSE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8753  0.8727  0.8669  0.8728  0.8799  0.8735  0.0042  
MSE (testset)     0.7661  0.7616  0.7516  0.7618  0.7742  0.7631  0.0073  
Fit time          2.21    2.21    2.13    2.22    2.03    2.16    0.07    
Test time         0.56    0.31    0.42    0.37    0.26    0.38    0.10    


{'test_rmse': array([0.87525444, 0.87271338, 0.8669412 , 0.87281018, 0.87989594]),
 'test_mse': array([0.76607034, 0.76162865, 0.75158704, 0.76179761, 0.77421686]),
 'fit_time': (2.2105743885040283,
  2.2073235511779785,
  2.130399465560913,
  2.2189676761627197,
  2.030393600463867),
 'test_time': (0.5574691295623779,
  0.30921292304992676,
  0.4210388660430908,
  0.3690519332885742,
  0.25927305221557617)}

## GridSearchCV 모델 성능 확인(최적 파라미터 탐색)

In [13]:
from surprise.model_selection import GridSearchCV

In [14]:
param={'n_factors':[20,40,60],'n_epochs':[50,100,200]}

In [16]:
gsc=GridSearchCV(SVD,param,measures=['rmse','mse'],cv=3)
gsc.fit(data)

## 결과 확인

In [21]:
gsc.best_score

{'rmse': 0.8920949554649763, 'mse': 0.7958662787042621}

In [22]:
gsc.best_params

{'rmse': {'n_factors': 20, 'n_epochs': 50},
 'mse': {'n_factors': 20, 'n_epochs': 50}}

성능이 가장 좋은 파라미터는 n_factors: 20, n_epochs :50  
성능이 가장 안좋은 파라미터는 n_factors: 20, n_epochs :200  

In [25]:
res=pd.DataFrame(gsc.cv_results).sort_values('rank_test_rmse')
res

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mse,split1_test_mse,split2_test_mse,mean_test_mse,std_test_mse,rank_test_mse,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_factors,param_n_epochs
0,0.897327,0.894843,0.884115,0.892095,0.005733,1,0.805196,0.800744,0.781659,0.795866,0.010209,1,2.363365,0.110477,0.635297,0.038034,"{'n_factors': 20, 'n_epochs': 50}",20,50
6,0.897957,0.893827,0.887718,0.893167,0.004206,2,0.806327,0.798926,0.788043,0.797765,0.007509,2,3.427660,0.114332,0.607859,0.079648,"{'n_factors': 60, 'n_epochs': 50}",60,50
3,0.903977,0.893992,0.887967,0.895312,0.006602,3,0.817175,0.799221,0.788486,0.801627,0.011835,3,2.882982,0.070417,0.678960,0.134231,"{'n_factors': 40, 'n_epochs': 50}",40,50
7,0.910030,0.904478,0.901382,0.905297,0.003578,4,0.828154,0.818080,0.812490,0.819575,0.006482,4,6.817339,0.040426,0.678080,0.139754,"{'n_factors': 60, 'n_epochs': 100}",60,100
8,0.910391,0.908531,0.903618,0.907513,0.002857,5,0.828812,0.825428,0.816525,0.823588,0.005182,5,13.566146,0.100715,0.668158,0.167331,"{'n_factors': 60, 'n_epochs': 200}",60,200
4,0.922660,0.918845,0.917318,0.919608,0.002247,6,0.851301,0.844276,0.841472,0.845683,0.004134,6,5.615290,0.067276,0.769273,0.079559,"{'n_factors': 40, 'n_epochs': 100}",40,100
1,0.936150,0.927620,0.926425,0.930065,0.004330,7,0.876376,0.860480,0.858262,0.865039,0.008067,7,4.673581,0.154950,0.590600,0.127297,"{'n_factors': 20, 'n_epochs': 100}",20,100
5,0.938208,0.930458,0.926556,0.931741,0.004842,8,0.880234,0.865752,0.858506,0.868164,0.009033,8,11.466479,0.040094,0.625486,0.079966,"{'n_factors': 40, 'n_epochs': 200}",40,200
2,0.960528,0.958252,0.956678,0.958486,0.001580,9,0.922613,0.918247,0.915232,0.918698,0.003030,9,9.164898,0.089957,0.628974,0.055603,"{'n_factors': 20, 'n_epochs': 200}",20,200
